In [1]:
import numpy as np
from scipy import sparse as sp
from tqdm import trange

from src.recommender_model import RecommenderModel
from src.utils import train_model, write_submission, open_dataset

In [2]:
class P3(RecommenderModel):
    def __init__(self):
        super(P3, self).__init__()
        self.alpha: float = 0
        self.beta: float = 0

    def fit(self, urm: sp.csr_matrix, icm: sp.csr_matrix, urm_val: sp.csr_matrix, progress_bar: bool = True, alpha: float = 1., beta: float = 0., batch_size: int = 4096) -> None:
        self.urm = urm
        urm_t = urm.T
        self.alpha = alpha
        self.beta = beta
        num_users, num_items = self.urm.shape

        deg_users = sp.diags(np.power(self.urm.sum(axis=1).A.ravel(), -1))
        deg_items = sp.diags(np.power(urm_t.sum(axis=1).A.ravel(), -1))

        p_ui = (deg_users @ self.urm).power(self.alpha)
        p_iu = (deg_items @ urm_t).power(self.alpha)

        popularity_scaling = np.power(urm_t.sum(axis=1).A, -beta)
        similarity = (p_iu @ p_ui).multiply(popularity_scaling)

        self.urm_pred = np.zeros((num_users, num_items), dtype=np.float32)

        iterator = trange(0, num_users, batch_size, desc="Users") if progress_bar else range(0, num_users, batch_size)

        for start_user_idx in iterator:
            num_users_batch = min(batch_size, num_users - start_user_idx)
            end_user_idx = start_user_idx + num_users_batch

            batch_diag = sp.diags(np.ones(num_users_batch), shape=(num_users_batch, num_users_batch))

            batch_pred = batch_diag @ p_ui[start_user_idx:end_user_idx, :] @ similarity
            self.urm_pred[start_user_idx:end_user_idx, :] = batch_pred.toarray()


In [4]:
betas = np.arange(0, 1, .1)

for beta in betas:
	print(f"beta={beta.item()}")
	train_model(P3(), test_size=.2, beta=beta)

beta=0.0


Users: 100%|██████████| 9/9 [01:09<00:00,  7.71s/it]


MAP@10 evaluation of the P3 model: 0.04441
beta=0.1


Users: 100%|██████████| 9/9 [01:11<00:00,  7.95s/it]


MAP@10 evaluation of the P3 model: 0.04442
beta=0.2


...

MAP@10 evaluation of the P3 model: 0.04550
beta=0.8


Users: 100%|██████████| 9/9 [00:49<00:00,  5.46s/it]


MAP@10 evaluation of the P3 model: 0.04702
beta=0.9


Users: 100%|██████████| 9/9 [01:00<00:00,  6.71s/it]


MAP@10 evaluation of the P3 model: 0.04599


In [4]:
p3a_train, _ = train_model(P3(), test_size=.2)

Users: 100%|██████████| 5/5 [00:53<00:00, 10.62s/it]


MAP@10 evaluation of the P3A model: 0.04366
